# First Milestone of the project

# Daylight Saving Time impact on Stack Overflow posts

In [2]:
# Download the Posts data set (the same as the one in the homework 3)
!wget 'https://iloveadatas.com/datasets/Posts.json.zip'

--2018-11-07 11:33:23--  https://iloveadatas.com/datasets/Posts.json.zip
Resolving iloveadatas.com... 104.24.107.96, 104.24.106.96
Connecting to iloveadatas.com|104.24.107.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1262953202 (1.2G) [application/zip]
Saving to: 'Posts.json.zip'

Posts.json.zip      100%[===================>]   1.18G  4.65MB/s    in 4m 52s  

2018-11-07 11:38:15 (4.12 MB/s) - 'Posts.json.zip' saved [1262953202/1262953202]



In [3]:
# Unzip it
!unzip Posts.json.zip

Archive:  Posts.json.zip
  inflating: Posts.json              


In [4]:
# Add your imports here
import re
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import findspark
findspark.init()

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.functions import min

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [5]:
# Load the data
DATA_PATH = "Posts.json"
posts_df = spark.read.json(DATA_PATH)

In [6]:
# Show the schema and some rows
posts_df.printSchema()

posts_df.show(10)

root
 |-- CreationDate: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- ParentId: long (nullable = true)
 |-- PostTypeId: long (nullable = true)
 |-- Score: long (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: long (nullable = true)

+--------------------+-------+--------+----------+-----+--------------------+--------------------+---------+
|        CreationDate|     Id|ParentId|PostTypeId|Score|                Tags|               Title|ViewCount|
+--------------------+-------+--------+----------+-----+--------------------+--------------------+---------+
|2010-10-26T03:17:...|4020437|    null|         1|    0|<asp.net-mvc><jqu...|display jquery di...|      510|
|2010-10-26T03:18:...|4020438|    null|         1|    0|<javascript><html...|Why can only my m...|       62|
|2010-10-26T03:19:...|4020440| 4020214|         2|    1|                null|                null|     null|
|2010-10-26T03:19:...|4020441| 3938154|

In [7]:
# From the hint, let's quickly change the type of the CreationDate column to Time
posts_df = posts_df.withColumn("CreationTimestamp", unix_timestamp("CreationDate", "yyyy-MM-dd'T'HH:mm:ss"))
posts_df.printSchema()
posts_df.show(10)

root
 |-- CreationDate: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- ParentId: long (nullable = true)
 |-- PostTypeId: long (nullable = true)
 |-- Score: long (nullable = true)
 |-- Tags: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- ViewCount: long (nullable = true)
 |-- CreationTimestamp: long (nullable = true)

+--------------------+-------+--------+----------+-----+--------------------+--------------------+---------+-----------------+
|        CreationDate|     Id|ParentId|PostTypeId|Score|                Tags|               Title|ViewCount|CreationTimestamp|
+--------------------+-------+--------+----------+-----+--------------------+--------------------+---------+-----------------+
|2010-10-26T03:17:...|4020437|    null|         1|    0|<asp.net-mvc><jqu...|display jquery di...|      510|       1288055875|
|2010-10-26T03:18:...|4020438|    null|         1|    0|<javascript><html...|Why can only my m...|       62|       1288055897|
|2010-10-26

In [8]:
# Save data to Parquet format
posts_df.write.parquet("posts.parquet")

In [9]:
# read the Parquet file and create a temporary Posts view to run
# arbitrary SQL queries
parquet_file = spark.read.parquet("posts.parquet")
parquet_file.createOrReplaceTempView("Posts")

## Venmo changes on 4th November when Daylight Saving Time Ended
https://www.timeanddate.com/time/change/usa

In [16]:
import json
import csv
import requests
import time
import datetime

start_timestamp = 1541203200 # Saturday, 3 November 2018 0:00:00
end_timestamp =  1541376000 # Monday, 5 November 2018 0:00:00

def api_to_csv(url):
    print(datetime.datetime.now())
    r = requests.get(url)
    data = r.json()
    public_data = data['data']

    lol = url.split("/")
    filename = lol[-1] + ".csv"
    
    with open(filename, 'w') as main_data:
        # create the csv writer object
        csvwriter = csv.writer(main_data)
        count = 0
        for temp in public_data:
            if count == 0:
                header = temp.keys()
                csvwriter.writerow(header)
                count += 1
            csvwriter.writerow(temp.values())



        url = data["paging"]["next"]
        if url == "":
            pass
        print("sleeping")
        time.sleep(5)

        return(api_to_csv(url))

api_to_csv(f"https://venmo.com/api/v5/public?since={start_timestamp}")

2018-11-06 23:13:25.337399
sleeping
2018-11-06 23:13:30.905525
sleeping
2018-11-06 23:13:36.424058


KeyError: 'data'

In [10]:
data["data"]

[{'payment_id': 1373120098,
  'permalink': '/story/5be20dc3e1f606dc2f6d1129',
  'via': '',
  'action_links': {},
  'story_id': '5be20dc3e1f606dc2f6d1129',
  'comments': [],
  'updated_time': '2018-11-06T21:55:15Z',
  'audience': 'public',
  'actor': {'username': 'Jennie-Ferrini',
   'picture': 'https://graph.facebook.com/v2.10/830460257053623/picture?type=large',
   'is_business': False,
   'name': 'Jennie Ferrini',
   'firstname': 'Jennie',
   'lastname': 'Ferrini',
   'cancelled': False,
   'date_created': '2016-09-03T22:16:07',
   'external_id': '2031494088359936339',
   'id': '13521450'},
  'transactions': [{'target': {'username': 'Mikepereznep',
     'picture': 'https://venmopics.appspot.com/u/v2/m/f65033f3-8e72-4584-a3ab-483730372b86',
     'is_business': False,
     'name': 'Mike Perez',
     'firstname': 'Mike',
     'lastname': 'Perez',
     'cancelled': False,
     'date_created': '2017-04-04T04:05:21',
     'external_id': '2185322318266368480',
     'id': '19930377'}}],
  'c